# 2. Mapper Configuration
http://docs.sqlalchemy.org/en/latest/orm/mapper_config.html

In [ ]:
%reset

In [ ]:
from sqlalchemy import create_engine
echo = True
engine = create_engine('sqlite:///:memory:', echo = echo)
engine

## Types of Mappings

Modern SQLAlchemy features two distinct styles of mapper configuration.  
1. The “Classical” style is SQLAlchemy’s original mapping API, whereas 
2. “Declarative” is the richer and more succinct system that builds on top of “Classical”.  

Both styles may be used interchangeably, as the end result of each is exactly the same - a user-defined class mapped by the mapper() function onto a ***selectable unit***, typically a ***Table***.

---
### Declarative Mapping

The Declarative Mapping is the typical way that mappings are constructed in modern SQLAlchemy. Making use of the Declarative system, the components of the user-defined class as well as the Table metadata to which the class is mapped are defined at once:

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey

Base = declarative_base()

class User(Base):
    __tablename__ = 'user'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)

Above, a basic single-table mapping with four columns.  
Additional attributes, such as relationships to other mapped classes, are also declared inline within the class definition:

In [ ]:
from sqlalchemy.orm import relationship

Base = declarative_base()

class User(Base):
    __tablename__ = 'user'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)

    addresses = relationship("Address", backref="user", order_by="Address.id")

class Address(Base):
    __tablename__ = 'address'

    id = Column(Integer, primary_key=True)
    user_id = Column(ForeignKey('user.id'))
    email_address = Column(String)

The declarative mapping system is introduced in the Object Relational Tutorial. For additional details on how this system works, see [Declarative](http://docs.sqlalchemy.org/en/latest/orm/extensions/declarative/index.html).

---
### Classical Mappings

A Classical Mapping refers to the configuration of a mapped class using the ***mapper()*** function, without using the Declarative system. This is SQLAlchemy’s original class mapping API, and is still the base mapping system provided by the ORM.

In “classical” form, the ***table metadata*** is created separately with the Table construct, then associated with the User class via the mapper() function:

In [ ]:
from sqlalchemy import Table, MetaData, Column, Integer, String, ForeignKey
from sqlalchemy.orm import mapper

metadata = MetaData()

user = Table('user', metadata,
             Column('id', Integer, primary_key=True),
             Column('name', String(50)),
             Column('fullname', String(50)),
             Column('password', String(12))
            )

class User(object):
    def __init__(self, name, fullname, password):
        self.name = name
        self.fullname = fullname
        self.password = password

mapper(User, user)

Information about mapped attributes, such as relationships to other classes, are provided via the ***properties*** dictionary. The example below illustrates a second Table object, mapped to a class called Address, then linked to User via relationship():

In [ ]:
from sqlalchemy.orm import clear_mappers

# metadata = MetaData()
address = Table('address', metadata,
                Column('id', Integer, primary_key=True),
                Column('user_id', Integer, ForeignKey('user.id')),
                Column('email_address', String(50))
               )

clear_mappers() # 清除mapping，重新 map一次
mapper(User, user, properties={
    'addresses' : relationship(Address, backref='user', order_by=address.c.id)
})

mapper(Address, address)

When using classical mappings, classes must be provided directly without the benefit of the ***“string lookup”*** system provided by Declarative. SQL expressions are typically specified in terms of the ***Table*** objects, i.e. ***address.c.id*** above for the Address relationship, and not Address.id, <font color='blue'>as Address may not yet be linked to table metadata, nor can we specify a string here</font>.

Some examples in the documentation still use the classical approach, but note that the classical as well as Declarative approaches are **fully interchangeable**. <font color='blue'>Both systems ultimately create the same configuration, consisting of a Table, user-defined class, linked together with a mapper()</font>. When we talk about “the behavior of mapper()”, this includes when using the Declarative system as well - it’s still used, just behind the scenes.

---
### Runtime Introspection of Mappings, Objects

The ***Mapper*** object is available from any mapped class, regardless of method, using the [Runtime Inspection API](http://docs.sqlalchemy.org/en/latest/core/inspection.html) system. Using the ***[inspect()](http://docs.sqlalchemy.org/en/latest/core/inspection.html#sqlalchemy.inspection.inspect)*** function, one can acquire the Mapper from a mapped class:

In [ ]:
from sqlalchemy import inspect
insp = inspect(User)
insp

Detailed information is available including ***Mapper.columns***:

In [ ]:
insp.columns

This is a namespace that can be viewed in a list format or via individual names:

In [ ]:
list(insp.columns)

In [ ]:
insp.columns.name

In [ ]:
insp.columns['name']

Other namespaces include ***Mapper.all_orm_descriptors***, which includes all mapped attributes as well as ***hybrids, association proxies***:

In [ ]:
insp.all_orm_descriptors

In [ ]:
insp.all_orm_descriptors.keys()

As well as Mapper.column_attrs:

In [ ]:
list(insp.column_attrs)

In [ ]:
insp.column_attrs.name

In [ ]:
insp.column_attrs.name.expression

## Mapping Columns and Expressions

The following sections discuss how ***table columns*** and ***SQL expressions*** are mapped to individual ***object attributes***.

### Mapping Table Columns

The default behavior of mapper() is to assemble all the ***columns*** in the mapped Table into mapped object ***attributes***, each of which are named according to the name of the column itself (specifically, the key attribute of Column). This behavior can be modified in several ways.

---
### Naming Columns Distinctly from Attribute Names

A mapping by default shares the same name for a Column as that of the mapped attribute - specifically it matches the ***Column.key*** attribute on Column, which by default is the same as the ***Column.name***.

The name assigned to the Python attribute which maps to Column can be ***different*** from either Column.name or Column.key just by assigning it that way, as we illustrate here in a Declarative mapping:

In [ ]:
Base = declarative_base()

class User(Base):
    __tablename__ = 'user'
    id = Column('user_id', Integer, primary_key=True)
    name = Column('user_name', String(50))

Where above User.id resolves to a column named user_id and User.name resolves to a column named user_name.

When mapping to an ***existing table***, the ***Column*** object can be referenced directly:

In [ ]:
Base = declarative_base() 

user_table = Table("user", Base.metadata)

class User(Base):
    __table__ = user_table
    id = user_table.c.user_id
    name = user_table.c.user_name

Or in a classical mapping, placed in the ***properties*** dictionary with the desired key:

In [ ]:
mapper(User, user_table, 
       properties={
                    'id': user_table.c.user_id,
                    'name': user_table.c.user_name,
                  }
      )

---
### Automating Column Naming Schemes from Reflected Tables

In the previous section Naming Columns Distinctly from Attribute Names, we showed how a Column explicitly mapped to a class can have a different attribute name than the column. But what if we aren’t listing out Column objects explicitly, and instead are ***automating the production of Table objects using reflection*** (e.g. as described in [Reflecting Database Objects](http://docs.sqlalchemy.org/en/latest/core/reflection.html))? In this case we can make use of the ***DDLEvents.column_reflect()*** event to intercept the production of Column objects and provide them with the ***Column.key*** of our choice:

In [ ]:
@event.listens_for(Table, "column_reflect")
def column_reflect(inspector, table, column_info):
    # set column.key = "attr_<lower_case_name>"
    column_info['key'] = "attr_%s" % column_info['name'].lower()

With the above event, the ***reflection of Column*** objects will be intercepted with our event that adds a new ”.key” element, such as in a mapping as below:

In [ ]:
class MyClass(Base):
    __table__ = Table("user", Base.metadata,
                autoload=True, autoload_with=engine)

If we want to qualify our event to only react for the specific MetaData object above, we can check for it in our event:

In [ ]:
@event.listens_for(Table, "column_reflect")
def column_reflect(inspector, table, column_info):
    if table.metadata is Base.metadata:
        # set column.key = "attr_<lower_case_name>"
        column_info['key'] = "attr_%s" % column_info['name'].lower()

---
### Naming All Columns with a Prefix

A quick approach to prefix column names, typically when mapping to an ***existing Table*** object, is to use column_prefix:

In [ ]:
class User(Base):
    __table__ = user_table
    __mapper_args__ = {'column_prefix':'_'}

The above will place attribute names such as _user_id, _user_name, _password etc. on the mapped User class.

This approach is ***uncommon*** in modern usage. For dealing with reflected tables, a more flexible approach is to use that described in [Automating Column Naming Schemes from Reflected Tables](http://docs.sqlalchemy.org/en/latest/orm/mapping_columns.html#mapper-automated-reflection-schemes).

---
### Using column_property for column level options

Options can be specified when mapping a Column using the ***column_property()*** function. This function explicitly creates the ***ColumnProperty*** used by the mapper() to keep track of the Column; normally, the mapper() creates this automatically. Using column_property(), we can pass additional arguments about how we’d like the Column to be mapped. Below, we pass an option ***active_history***, which specifies that a change to this column’s value should result in the former value being loaded first:

In [ ]:
from sqlalchemy.orm import column_property

class User(Base):
    __tablename__ = 'user'

    id = Column(Integer, primary_key=True)
    name = column_property(Column(String(50)), active_history=True)

***column_property()*** is also used to ***map a single attribute to multiple columns***. This use case arises when mapping to a ***join()*** which has attributes which are equated to each other:

In [ ]:
class User(Base):
    __table__ = user.join(address)

    # assign "user.id", "address.user_id" to the
    # "id" attribute
    id = column_property(user_table.c.id, address_table.c.user_id)

For more examples featuring this usage, see [Mapping a Class against Multiple Tables](http://docs.sqlalchemy.org/en/latest/orm/nonstandard_mappings.html#maptojoin).

Another place where column_property() is needed is to specify ***SQL expressions as mapped attributes***, such as below where we create an attribute fullname that is the string ***concatenation*** of the firstname and lastname columns:

In [ ]:
class User(Base):
    __tablename__ = 'user'
    id = Column(Integer, primary_key=True)
    firstname = Column(String(50))
    lastname = Column(String(50))
    fullname = column_property(firstname + " " + lastname)

sqlalchemy.orm.column_property(*columns, **kwargs)

> Provide a column-level property for use with a Mapper.

> Column-based properties can normally be applied to the mapper’s properties dictionary using the Column element directly. Use this function when the given column is not directly present within the mapper’s selectable; examples include SQL expressions, functions, and scalar SELECT queries.

> Columns that aren’t present in the mapper’s selectable won’t be persisted by the mapper and are effectively “read-only” attributes.

---
### Mapping a Subset of Table Columns

Sometimes, a Table object was made available using the reflection process described at [Reflecting Database Objects](http://docs.sqlalchemy.org/en/latest/core/reflection.html#metadata-reflection) to load the table’s structure from the database. For such a table that has lots of columns that don’t need to be referenced in the application, the ***include_properties*** or ***exclude_propertie***s arguments can specify that only a subset of columns should be mapped. For example:

In [ ]:
class User(Base):
    __table__ = user_table
    __mapper_args__ = {
        'include_properties' :['user_id', 'user_name']
    }

...will map the User class to the user_table table, only including the user_id and user_name columns - the rest are not referenced. Similarly:

In [ ]:
class Address(Base):
    __table__ = address_table
    __mapper_args__ = {
        'exclude_properties' : ['street', 'city', 'state', 'zip']
    }

...will map the Address class to the address_table table, including all columns present ***except*** street, city, state, and zip.

When this mapping is used, ***the columns that are not included will not be referenced in any SELECT statements emitted by Query***, nor will there be any mapped attribute on the mapped class which represents the column; assigning an attribute of that name will have no effect beyond that of a normal Python attribute assignment.

In some cases, multiple columns may have the ***same name***, such as when mapping to a ***join*** of two or more tables that share some column name. include_properties and exclude_properties can also accommodate Column objects to more accurately describe which columns should be included or excluded:

In [ ]:
class UserAddress(Base):
    __table__ = user_table.join(addresses_table)
    __mapper_args__ = {
        'exclude_properties' :[address_table.c.id],
        'primary_key' : [user_table.c.id]
    }

**Note**: insert and update defaults configured on individual Column objects, i.e. those described at Column Insert/Update Defaults including those configured by the default, update, server_default and server_onupdate arguments, will continue to function normally even if those Column objects are not mapped. This is because in the case of default and update, ***the Column object is still present on the underlying Table***, thus allowing the default functions to take place when the ORM emits an INSERT or UPDATE, and in the case of server_default and server_onupdate, the relational database itself maintains these functions.

### SQL Expressions as Mapped Attributes
Attributes on a mapped class can be linked to SQL expressions, which can be used in queries.

---
### Using a Hybrid

The easiest and most flexible way to link relatively simple SQL expressions to a class is to use a so-called ***“hybrid attribute”***, described in the section [Hybrid Attributes](http://docs.sqlalchemy.org/en/latest/orm/extensions/hybrid.html). The hybrid provides for an expression that works at both the Python level as well as at the SQL expression level. For example, below we map a class User, containing attributes firstname and lastname, and include a ***hybrid*** that will provide for us the fullname, which is the string concatenation of the two:

In [ ]:
from sqlalchemy.ext.hybrid import hybrid_property

class User(Base):
    __tablename__ = 'user'
    id = Column(Integer, primary_key=True)
    firstname = Column(String(50))
    lastname = Column(String(50))

    @hybrid_property
    def fullname(self):
        return self.firstname + " " + self.lastname

Above, the fullname attribute is interpreted ***at both the instance and class level***, so that it is available from an instance:

In [ ]:
some_user = session.query(User).first()
print some_user.fullname

as well as usable within queries:

In [ ]:
some_user = session.query(User).filter(User.fullname == "John Smith").first()

The string concatenation example is a simple one, where the Python expression can be dual purposed at the instance and class level. Often, the SQL expression must be distinguished from the Python expression, which can be achieved using ***hybrid_property.expression()***. Below we illustrate the case where a conditional needs to be present inside the hybrid, using the if statement in Python and the ***sql.expression.case()*** construct for SQL expressions:

In [ ]:
from sqlalchemy.ext.hybrid import hybrid_property
from sqlalchemy.sql import case

class User(Base):
    __tablename__ = 'user'
    id = Column(Integer, primary_key=True)
    firstname = Column(String(50))
    lastname = Column(String(50))

    @hybrid_property
    def fullname(self):
        if self.firstname is not None:
            return self.firstname + " " + self.lastname
        else:
            return self.lastname

    @fullname.expression
    def fullname(cls):
        return case([
            (cls.firstname != None, cls.firstname + " " + cls.lastname),
        ], else_ = cls.lastname)

---
### Using column_property

The ***orm.column_property()*** function can be used to map a SQL expression in a manner similar to a regularly mapped Column. With this technique, the attribute is loaded along with all other column-mapped attributes at load time. This is in some cases an advantage over the usage of hybrids, as ***the value can be loaded up front at the same time as the parent row of the object***, particularly if the expression is one which links to other tables (typically as a ***correlated subquery***) to access data that wouldn’t normally be available on an already loaded object.

Disadvantages to using orm.column_property() for SQL expressions include that the expression must be compatible with the SELECT statement emitted for the class as a whole, and there are also some configurational quirks which can occur when using orm.column_property() from declarative mixins.

Our “fullname” example can be expressed using orm.column_property() as follows: